In [63]:
import math
import metapy
import pytoml
import csv
import pandas as pd


In [64]:
df = pd.read_csv ('/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_lessons.csv')
print(df.head())

  course_id  week_nbr  video_id  \
0    cs-410         1         1   
1    cs-410         1         2   
2    cs-410         1         3   
3    cs-410         1         4   
4    cs-410         1         5   

                                             content  
0  [SOUND] >> This lecture is about Natural Langu...  
1  [SOUND] In this lecture,we're going to talk ab...  
2  [MUSIC] This lecture is aboutthe text retrieva...  
3  [SOUND] This lecture is a overview oftext retr...  
4  [SOUND] This lecture is about thevector space ...  


In [65]:
print(df.columns)

Index(['course_id', 'week_nbr', 'video_id', 'content'], dtype='object')


In [66]:
segcolumns = ['content']

In [67]:
new_df_text = pd.DataFrame(df, columns=segcolumns)

In [68]:
print(new_df_text)

                                              content
0   [SOUND] >> This lecture is about Natural Langu...
1   [SOUND] In this lecture,we're going to talk ab...
2   [MUSIC] This lecture is aboutthe text retrieva...
3   [SOUND] This lecture is a overview oftext retr...
4   [SOUND] This lecture is about thevector space ...
..                                                ...
68  [MUSIC] This lecture is aboutevaluation of tex...
69  [SOUND] This lecture is about text categorizat...
70  [SOUND] This lecture isa continued discussion ...
71  [SOUND]This lecture is aboutthe contextual tex...
72  This lecture is a summaryof this whole course....

[73 rows x 1 columns]


In [69]:
print(new_df_text)

                                              content
0   [SOUND] >> This lecture is about Natural Langu...
1   [SOUND] In this lecture,we're going to talk ab...
2   [MUSIC] This lecture is aboutthe text retrieva...
3   [SOUND] This lecture is a overview oftext retr...
4   [SOUND] This lecture is about thevector space ...
..                                                ...
68  [MUSIC] This lecture is aboutevaluation of tex...
69  [SOUND] This lecture is about text categorizat...
70  [SOUND] This lecture isa continued discussion ...
71  [SOUND]This lecture is aboutthe contextual tex...
72  This lecture is a summaryof this whole course....

[73 rows x 1 columns]


In [70]:
corpus = []
for index, row in df.iterrows():
    text = row['content']
    corpus.append(text)

In [71]:
with open('course.dat', 'w') as f:
    for text in corpus:
        f.write(text)
        f.write('\n')

In [72]:
idx = metapy.index.make_inverted_index('config.toml')

In [73]:
idx.num_docs()

74

In [74]:
idx.unique_terms()

7312

In [75]:
idx.avg_doc_length()

1511.648681640625

In [76]:
idx.total_corpus_terms()

111862

In [77]:
ranker = metapy.index.OkapiBM25()

In [89]:
query = metapy.index.Document()
query.content('PLSA')

In [90]:
top_docs = ranker.score(idx, query, num_results=5)
top_docs

[(68, 6.3648810386657715), (67, 5.483545780181885), (73, 5.0520219802856445)]

In [91]:
for num, (d_id, _) in enumerate(top_docs):
    print(idx.metadata(d_id).get(;))


TypeError: get(): incompatible function arguments. The following argument types are supported:
    1. (self: metapy.metapy.index.Metadata, arg0: str) -> object

Invoked with: <metapy.metapy.index.Metadata object at 0x7f98752790f0>

In [54]:
stri = '/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_lessons.csv'
print(stri)

/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/courseera_video_lessons.csv


In [55]:
print(stri.rfind('/'))

61


In [57]:
print(stri[0: stri.rfind('/')+1])

/Users/kartikpatel/Desktop/CS 410/Final Project/CourseProject/
